In [2]:
import os,sys
from glob import glob
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import pickle


img_dir = "/root/autoencoder_denoiser/dataset/real_img_referral_for_testing"
clean_dir = os.path.join(img_dir, "real_hsqc_clean")
noisy_dir = os.path.join(img_dir, "real_hsqc_noisy")


all_img_paths = glob(noisy_dir+"/*")
all_img_paths

['/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Quercetin_HSQC_noisy_4.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Quercetin_noisy_5.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Quercetin_noisy_3.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Palmitic acid_noisy_3.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Menthol_noisy_7.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Cyclosporin A_noisy_5.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Reserpine_noisy_2.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Quercetin_HSQC_noisy_1.png',
 '/root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Erythromycin_noisy_5.png',
 '/root/autoencode

In [5]:
import random
eighty_percent = int(len(all_img_paths)*0.8)
nighty_percent = int(len(all_img_paths)*0.9)
random.shuffle(all_img_paths)
train_img_paths = all_img_paths[:eighty_percent]
val_img_paths = all_img_paths[eighty_percent:nighty_percent]
test_img_paths = all_img_paths[nighty_percent:]

train_imgs=[]
val_imgs=[]
test_imgs = []
##### train split
for splitted_imgs, splitted_img_paths, split in [(train_imgs, train_img_paths, 'train')   ]:
    for stage in range(6,0,-1):
        print(f'stage : {stage}')
        for img_path in tqdm(splitted_img_paths):
            noise_level = img_path.split('.')[0].split('_')[-1]

            in_range = stage == int(noise_level) or (stage ==6 and int(noise_level)>=6)
            if in_range:
                print(f'processing : {img_path}')
                '''noise'''
                img = Image.open(img_path)
                img = np.array(img)
                plus = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
                minus = 1-img[:,:, 2]/255
                # print(plus.shape)
                img_result = plus-minus
                resized_input = cv2.resize(img_result.astype("float32"), (120, 180))
                '''ground truth'''
                ground_path = img_path.replace("real_hsqc_noisy","real_hsqc_clean").replace("_noisy","_original")
                ground_path = ground_path[:-5] + '1.png'
                if ground_path[-6].isdigit():
                    # print(ground_path)
                    ground_path = ground_path[:-6]+ground_path[-5:]
                # print(gound_path)
                img = Image.open(ground_path)
                img = np.array(img)
                plus_groud = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
                minus_groud = 1-img[:,:, 2]/255
                ground_truth = plus_groud - minus_groud
                ground_truth = cv2.resize(ground_truth.astype("float32"), (120, 180))
                splitted_imgs.append((resized_input, ground_truth))
        np.save(f'/root/autoencoder_denoiser/dataset/save_real_imgs_in_stages/{split}_stage_{stage}.npy', splitted_imgs, allow_pickle=True)
        print(f'saving {split}_stage_{stage}')
        

stage : 6


  0%|          | 0/52 [00:00<?, ?it/s]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Erythromycin_noisy_6.png


  2%|▏         | 1/52 [00:03<02:50,  3.34s/it]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Artemisinin_noisy_6.png


...

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Quercetin_noisy_1.png


 96%|█████████▌| 50/52 [00:42<00:01,  1.13it/s]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Palmitic acid_noisy_1.png


100%|██████████| 52/52 [00:47<00:00,  1.09it/s]

saving train_stage_1


In [9]:
##### val , test split
for splitted_imgs, splitted_img_paths, split in [(val_imgs, val_img_paths, 'val'), 
                                                 (test_imgs, test_img_paths, 'test')]:
 
        for img_path in tqdm(splitted_img_paths):

                print(f'processing : {img_path}')
                '''noise'''
                img = Image.open(img_path)
                img = np.array(img)
                plus = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
                minus = 1-img[:,:, 2]/255
                # print(plus.shape)
                img_result = plus-minus
                resized_input = cv2.resize(img_result.astype("float32"), (120, 180))
                '''ground truth'''
                ground_path = img_path.replace("real_hsqc_noisy","real_hsqc_clean").replace("_noisy","_original")
                ground_path = ground_path[:-5] + '1.png'
                if ground_path[-6].isdigit():
                    # print(ground_path)
                    ground_path = ground_path[:-6]+ground_path[-5:]
                # print(gound_path)
                img = Image.open(ground_path)
                img = np.array(img)
                plus_groud = 1-img[:,:, 0]/255 # I will assume this is plus but not sure
                minus_groud = 1-img[:,:, 2]/255
                ground_truth = plus_groud - minus_groud
                ground_truth = cv2.resize(ground_truth.astype("float32"), (120, 180))
                splitted_imgs.append((resized_input, ground_truth))
        np.save(f'/root/autoencoder_denoiser/dataset/save_real_imgs_in_stages/{split}.npy', splitted_imgs, allow_pickle=True)
        print(f'saving {split}_stage_{stage}')

stage : 10


  0%|          | 0/65 [00:00<?, ?it/s]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Erythromycin_noisy_10.png


100%|██████████| 65/65 [00:05<00:00, 11.00it/s]


stage : 9


...

 18%|█▊        | 12/65 [00:05<00:25,  2.06it/s]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Erythromycin_noisy_6.png


 46%|████▌     | 30/65 [00:11<00:13,  2.68it/s]

processing : /root/autoencoder_denoiser/dataset/real_img_referral_for_testing/real_hsqc_noisy/Menthol_noisy_6.png


100%|██████████| 65/65 [00:17<00:00,  3.67it/s]


In [10]:
for i in range(1, 7):
    with open(f'/root/autoencoder_denoiser/dataset/save_real_imgs_in_stages/stage_{i}.pkl', 'rb') as f:
        img_loaded =  pickle.load(f)
    print(len(img_loaded))
        

65
53
41
29
17
8


In [ ]:
'''data leak'''
'''first split into train val test, and then multistage'''